In [28]:
import torch
import torch.nn.functional as fun
import numpy as np
from google_play_scraper import Sort, reviews
import csv
from tqdm import tqdm

from nltk.tokenize import wordpunct_tokenize

def clean_word(sentence):
    sentence = wordpunct_tokenize(sentence)
    cleaned = []
    for word in sentence:
        if(word.isalpha()):
            cleaned.append(word.lower())
    return cleaned

def get_corpus_from_csv():
    corpus = []
    with open("dfyutup-cleanedfixversion3.csv") as file_buffer:
        csv_reader = list(csv.reader(file_buffer))
        n = len(csv_reader)
        for i in range(1, n):
            corpus.append(clean_word(csv_reader[i][2]))
    return corpus

    
def get_corpus_google_play(cnt = 2000):
    result, _ = reviews(
        'com.miHoYo.GenshinImpact',
        lang='id', 
        country='id', 
        sort=Sort.MOST_RELEVANT, 
        count=cnt, 
        filter_score_with=None
    )
    return [clean_word(konten["content"]) for konten in result]

# corpus = []
# corpus = get_corpus_google_play()
# print(corpus)
# print(get_corpus_from_csv())

# corpus = get_corpus_from_csv()
# print(corpus)

In [7]:
import pickle

# obj0, obj1, obj2 are created here...

def save_corpus(idx = 0):
    # Saving the objects:
    global corpus
    with open(f'corpus-objs-{idx}.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([corpus], f)

def load_corpus(idx = 0):
    # Getting back the objects:
    global corpus
    save_corpus(999)
    with open(f'corpus-objs-{idx}.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
       [corpus] = pickle.load(f)

In [29]:
load_corpus(1)

In [150]:
print(corpus[30:50])

[['sungguh', 'kadang', 'aku', 'gagal', 'focus', 'apa', 'aku', 'ini', 'bermain', 'game', 'atau', 'nonton', 'drama', 'semua', 'karna', 'background', 'music', 'nya', 'yg', 'super', 'duper', 'enak', 'di', 'telinga', 'blm', 'lg', 'pengisi', 'suara', 'jepang', 'yg', 'khas', 'bikin', 'betah', 'mantengin', 'story', 'sudah', 'hampir', 'gak', 'sia', 'sia', 'aku', 'download', 'bermain', 'game', 'ini', 'puas', 'dalam', 'segala', 'hal', 'meski', 'terkadang', 'sulit', 'tp', 'aku', 'menikmati', 'sangat', 'suka', 'genshin', 'impact'], ['game', 'nya', 'memang', 'sudah', 'bagus', 'dari', 'segi', 'mana', 'pun', 'tapi', 'setelah', 'update', 'jadi', 'ngelag', 'banget', 'padahal', 'sebelumnya', 'ga', 'separah', 'ini', 'kalau', 'bisa', 'game', 'nya', 'lebih', 'di', 'optimalkan', 'lagi', 'atau', 'tambahkan', 'fitur', 'render', 'distance', 'karena', 'memang', 'untuk', 'render', 'map', 'nya', 'bisa', 'di', 'bilang', 'terlalu', 'jauh', 'mungkin', 'itu', 'penyebab', 'lag', 'nya', 'jadi', 'mohon', 'untuk', 'pihak'

In [30]:
import torch
import torch.nn.functional as fun
import numpy as np


def create_vocab(corpus, limit_occ = 20):
    """
    membuat vocabulary, dengan 2 info penting: 
        1. word_id -> mapping kata ke id (int)
        2. id_word -> sebaliknya, mapping dari id ke word
    """
    word_id = {}
    id_word = []

    occurence = dict()
    i = 0
    for sentence in corpus:
        for word in sentence:
            occurence.setdefault(word, 0)
            occurence[word] += 1
            
    for k, v in occurence.items():
        if(v < limit_occ): continue
        print(k)
        word_id[k] = i
        id_word.append(k)
        i += 1
                
    id_word = {id: word for (word, id) in word_id.items()}
    return word_id, id_word

print(len(corpus))
word_id, id_word = create_vocab(corpus)
vocab_size = len(word_id)

print(word_id)


2000
grafis
yang
memanjakan
mata
quest
dan
event
sangat
banyak
map
open
world
luas
untuk
atau
lagi
bosen
char
jadi
cocok
buat
para
selain
itu
ada
fitur
pc
mobile
saat
masih
bisa
main
akun
di
kita
masalah
ketika
saya
sedang
story
waktu
skip
dialog
pas
domain
tiba
tidak
bahkan
sudah
kali
tetap
mau
jalan
juga
lumayan
bagus
jujur
menurut
aku
genshin
impact
game
dari
segi
nya
yah
walaupun
beberapa
bikin
mungkin
harus
soalnya
player
pasti
bosan
dengan
konten
kayak
gitu
terus
aja
gue
the
character
tapi
gak
suka
ini
gamenya
terlalu
besar
puzzle
agak
susah
kalau
karakter
kadang
membuat
banget
grafik
ga
pun
cuma
kekurangan
yaitu
berat
ganti
device
seru
gameplay
menarik
tolong
tambahkan
dapat
teka
teki
unik
serta
petualangan
keren
pengalaman
bermain
low
paling
yg
pernah
mainin
android
kurang
terkadang
sama
misi
bgt
belum
kalo
jaringan
error
udah
lama
awal
karna
server
lah
ya
jangan
satu
ad
gacha
selalu
ampas
d
pity
orang
lain
aj
udh
resin
artefak
jelek
up
gk
sih
mainkan
sampe
hp
setiap
dapet
mala

In [31]:
# dataset generator
def iter_dataset(corpus, context_window=2):
    for sentence in corpus:
        words = sentence
        for i in range(context_window, len(words)):
            kiri = i - context_window
            kanan = i + context_window
            if(kiri < 0 or kanan >= len(words)):
                continue
            center = words[i]
            context = words[kiri:i] + words[i + 1:kanan + 1]
            # print(kiri, kanan, center, context)
            yield center, context

def data_to_tensor(center, context):
    if(center not in word_id.keys()):
        return 0, -1, []
    can = 1
    for word in context:
        if(word not in word_id.keys()):
            can = 0
            break
    if(not can):
        return 0, -1, []
    return\
    1,\
    fun.one_hot(torch.tensor([word_id[center]]), num_classes=vocab_size).float(),\
    fun.one_hot(torch.tensor([word_id[word] for word in context]), num_classes=vocab_size).float()


def get_dataset(corpus, context_window = 2):
    # membuat dataset
    X = torch.tensor([])
    Y = torch.tensor([])
    averager = torch.tensor(np.full((1, context_window * 2), 1/(context_window * 2))).float()
    for center, context in tqdm(list(iter_dataset(corpus, context_window))):
        # print(center, context)
        can, tmp_center, tmp_context = data_to_tensor(center, context)
        if(can == 0):
            continue
        # print(tmp_context)
        averaged = averager.matmul(tmp_context)
        X = torch.cat((X, averaged), dim = 0)
        Y = torch.cat((Y, tmp_center), dim = 0)
        # break
    print(X.shape, Y.shape)
    return X, Y

context_list, center_list = get_dataset(corpus)


100%|██████████| 62396/62396 [04:29<00:00, 231.12it/s] 

torch.Size([18462, 530]) torch.Size([18462, 530])


In [35]:
import pickle

# obj0, obj1, obj2 are created here...

def save_training(idx = 0):
    # Saving the objects:
    global center_list, context_list
    with open(f'training-objs-{idx}.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([center_list, context_list], f)

def load_training(idx = 0):
    # Getting back the objects:
    global center_list, context_list
    save_training(999)
    with open(f'training-objs-{idx}.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
       center_list, context_list = pickle.load(f)

In [39]:
save_training(1)

In [40]:
# Softmax
def softmax(x):
    maxes = torch.max(x, 1, keepdim=True)[0]
    x_exp = torch.exp(x - maxes)
    x_exp_sum = torch.sum(x_exp, 1, keepdim=True)
    return x_exp / x_exp_sum

# Categorical Cross Entropy Loss


def CCEloss(Y_pred, Y_true):
    m = Y_pred.size()[0]
    return -(1 / m) * torch.sum(Y_true * torch.log(Y_pred))
    
tmp_c = None
tmp_w = None

In [337]:

def train(training_X, training_Y, EPOCHS = 200, load = False):
    global tmp_w
    global tmp_c

    EMBEDDING_DIMS = 10

    initrange = 1

    # trick: di awal, mode gradient jangan diaktifkan dahulu, karena ingin
    # memodifikasi nilai di C dan W secara in-place
    C = torch.rand(vocab_size, EMBEDDING_DIMS, requires_grad = False)
    W = torch.rand(EMBEDDING_DIMS, vocab_size, requires_grad = False)

    # agar nilai awal parameter di C dan W berkisar di antara -initrange hingga +initrange
    C = -2 * initrange * C + initrange
    W = -2 * initrange * W + initrange

    # setelah operasi in-place modification, baru kita set mode gradient
    C.requires_grad = True
    W.requires_grad = True
    if(load):
        C = tmp_c
        W = tmp_w

    print(f'C shape is: {C.shape}, W shape is: {W.shape}')
    print(C)
    print(W)

    LEARNING_RATE = 30
    LR_DECAY = 0.99

    for i in range(EPOCHS):
        X = training_X
        Y = training_Y

        h = X.mm(C)
        Y_pred = softmax(h.mm(W))

        loss = CCEloss(Y_pred, Y)
        loss.backward()

        # update C dan W
        with torch.no_grad():
            C -= LEARNING_RATE * C.grad
            W -= LEARNING_RATE * W.grad
            
            C.grad.zero_()
            W.grad.zero_()
        tmp_w = W
        tmp_c = C
        if i % 10 == 0:
            w_1 = "gue"
            w_2 = "gua"
            vector_1 = C[word_id[w_1]]
            vector_2 = C[word_id[w_2]]
            sim = fun.cosine_similarity(vector_1, vector_2, dim = 0)
            print(f'Epoch {i}, loss = {loss}, sim({w_1}, {w_2}) = {sim}')
    return C, W


In [368]:

embedding_C, context_W = train(center_list, context_list, 5000, True)

C shape is: torch.Size([530, 10]), W shape is: torch.Size([10, 530])
tensor([[ 0.5715, -0.0058,  0.0583,  ..., -1.0425,  0.0690, -0.9610],
        [-0.0082,  0.1251,  0.6803,  ...,  0.5822,  0.4412, -0.0718],
        [-0.1930, -1.1469, -0.8664,  ..., -0.9197,  0.0242, -0.4711],
        ...,
        [ 0.2813, -0.2737, -0.2538,  ...,  1.1345,  0.5771, -0.7115],
        [-0.6530,  0.2484, -0.0941,  ...,  0.5894,  0.8503, -0.6690],
        [ 0.3126, -0.7185, -0.1139,  ...,  0.2824,  0.9906, -0.6022]],
       requires_grad=True)
tensor([[-0.1960,  0.2329,  0.7937,  ..., -0.3576, -0.6687,  1.1355],
        [ 0.4149, -0.8686,  0.0684,  ..., -0.3058,  0.6380,  0.2099],
        [ 0.5036,  0.6650,  0.2827,  ..., -0.6293, -0.3996,  0.9500],
        ...,
        [-0.2752,  0.9083, -0.5180,  ..., -0.1052,  0.3146, -0.3136],
        [-0.3913,  1.8311, -0.5870,  ..., -0.3757,  0.0655, -0.2484],
        [ 0.1507, -1.9801,  1.0850,  ...,  0.5920,  0.4988,  1.1263]],
       requires_grad=True)
Epoch 0, 

KeyboardInterrupt: 

In [45]:
import pickle

# obj0, obj1, obj2 are created here...
tmp_w = []
tmp_c = []
def save(idx = 0):
    # Saving the objects:
    global tmp_w, tmp_c
    with open(f'cbow-objs-{idx}.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([tmp_w, tmp_c], f)

def load(idx = 0):
    # Getting back the objects:
    global tmp_w, tmp_c
    save(999)
    with open(f'cbow-objs-{idx}.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
        tmp_w, tmp_c = pickle.load(f)

In [367]:
save(1)

In [381]:
def get_diff(word_1, word_2):
    global tmp_c
    vector_1 = tmp_c[word_id[word_1]]
    vector_2 = tmp_c[word_id[word_2]]
    print(fun.cosine_similarity(vector_1, vector_2, dim = 0))

def show_vector(word_1):
    global tmp_c
    vector_1 = tmp_c[word_id[word_1]].detach().numpy()
    print(vector_1)


def predict(context, K = 30, context_window = 2):
    global tmp_c
    global tmp_w
    print()
    print(f"Predicting kata di antara \"{context[:context_window]}\"  \"{context[context_window:]}\"")
    C = tmp_c
    W = tmp_w
    averager = torch.tensor(np.full((1, context_window * 2), 1/(context_window * 2))).float()
    tmp_context = fun.one_hot(torch.tensor([word_id[word] for word in context]), num_classes=vocab_size).float()

    averaged = averager.matmul(tmp_context)
    h = averaged.mm(C)
    Y_pred = softmax(h.mm(W))[0]
    prediction = sorted([(Y_pred[i], i) for i in range(len(Y_pred))], reverse=True)
    # print(prediction)
    printed = 0
    stop_words = {"saat", "udah", "buat", "aja", "padahal", "banget", "bagus", "setelah", "sering", "sangat", "lebih", "main", "game", "sih", "banyak", "tidak", "pas", "karena", "ya", "sudah", "dari", "juga", "di", "saya", "nya", "tapi", "dan", "yang", "untuk", "ada", "lagi", "bisa", "ini", "yg", "itu", "ga", "masih", "gak", "kalau", "jadi"}
    for word in context:
        stop_words.add(word)
    for pred in prediction:
        if(printed == K): break
        if(id_word[pred[1]] in stop_words): continue
        printed += 1
        print(id_word[pred[1]], pred[0])
    # print(prediction)
    # print(Y_pred)


In [392]:
predict(["developer", "segera", "tolong", "tambahin"], 7)


Predicting kata di antara "['developer', 'segera']"  "['tolong', 'tambahin']"
bug tensor(0.0197, grad_fn=<SelectBackward0>)
perbaiki tensor(0.0169, grad_fn=<SelectBackward0>)
update tensor(0.0159, grad_fn=<SelectBackward0>)
diperbaiki tensor(0.0123, grad_fn=<SelectBackward0>)
masalah tensor(0.0115, grad_fn=<SelectBackward0>)
fitur tensor(0.0102, grad_fn=<SelectBackward0>)
mohon tensor(0.0093, grad_fn=<SelectBackward0>)


In [388]:
predict(["update", "hp", "terlalu", "besar"], 5)


Predicting kata di antara "['update', 'hp']"  "['terlalu', 'besar']"
tolong tensor(0.0182, grad_fn=<SelectBackward0>)
jangan tensor(0.0074, grad_fn=<SelectBackward0>)
berat tensor(0.0070, grad_fn=<SelectBackward0>)
lama tensor(0.0066, grad_fn=<SelectBackward0>)
agak tensor(0.0058, grad_fn=<SelectBackward0>)


In [385]:
predict(["genshin", "suka", "close", "sendiri"], 5)


Predicting kata di antara "['genshin', 'suka']"  "['close', 'sendiri']"
aku tensor(0.0110, grad_fn=<SelectBackward0>)
force tensor(0.0080, grad_fn=<SelectBackward0>)
bermain tensor(0.0079, grad_fn=<SelectBackward0>)
hanya tensor(0.0070, grad_fn=<SelectBackward0>)
grafik tensor(0.0065, grad_fn=<SelectBackward0>)


In [386]:
predict(["mihoyo", "tolong", "primogem", "primo"], 5)


Predicting kata di antara "['mihoyo', 'tolong']"  "['primogem', 'primo']"
kasih tensor(0.0209, grad_fn=<SelectBackward0>)
bintang tensor(0.0196, grad_fn=<SelectBackward0>)
terlalu tensor(0.0130, grad_fn=<SelectBackward0>)
semoga tensor(0.0108, grad_fn=<SelectBackward0>)
kedepannya tensor(0.0078, grad_fn=<SelectBackward0>)


In [390]:
predict(["dari", "segi", "sangat", "keren"], 10)


Predicting kata di antara "['dari', 'segi']"  "['sangat', 'keren']"
grafik tensor(0.0431, grad_fn=<SelectBackward0>)
alur tensor(0.0173, grad_fn=<SelectBackward0>)
cerita tensor(0.0162, grad_fn=<SelectBackward0>)
story tensor(0.0141, grad_fn=<SelectBackward0>)
karakter tensor(0.0133, grad_fn=<SelectBackward0>)
gameplay tensor(0.0113, grad_fn=<SelectBackward0>)
seru tensor(0.0103, grad_fn=<SelectBackward0>)
gamenya tensor(0.0101, grad_fn=<SelectBackward0>)
grafis tensor(0.0087, grad_fn=<SelectBackward0>)
menarik tensor(0.0081, grad_fn=<SelectBackward0>)


In [391]:
show_vector("gua")
show_vector("gue")
show_vector("aku")
show_vector("gw")
get_diff("gua", "gw")
get_diff("gue", "gw")
get_diff("yang", "yg")
get_diff("ga", "gk")
get_diff("sangat", "banget")
get_diff("sangat", "terlalu")
get_diff("mohon", "tolong")

[-0.33343107  0.3071951   0.30983156  0.7165906   0.01492563  0.61837626
  0.22355306  0.9845062  -0.38135502 -0.82050145]
[-0.42533067 -0.6217535  -0.6451028  -0.13565081 -0.5717958   0.02543749
  0.06367031  1.7151709  -0.5196276  -1.9197856 ]
[-0.25876036  0.45935193  0.42748362  0.32185405 -0.44205353  0.67842907
 -0.23652549  0.44462824 -0.02828242 -1.1113    ]
[ 0.34546953  0.03561064 -0.36204     0.44171232 -0.09715364  0.4253653
 -0.05124992  0.7585656  -0.34862667 -1.2416128 ]
tensor(0.7608, grad_fn=<SumBackward1>)
tensor(0.8081, grad_fn=<SumBackward1>)
tensor(0.9637, grad_fn=<SumBackward1>)
tensor(0.9280, grad_fn=<SumBackward1>)
tensor(0.5022, grad_fn=<SumBackward1>)
tensor(0.5312, grad_fn=<SumBackward1>)
tensor(0.8722, grad_fn=<SumBackward1>)
